In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

In [2]:
ff_factors = pd.read_csv('data/F-F_Research_Data_Factors_Monthly.csv',index_col=0)
ff_factors.index = pd.to_datetime(ff_factors.index)
ff_cov_df = ff_factors.ewm(halflife=36).cov()
ff_cov_df.to_pickle(f'data/ff_cov_halflife{36}.pkl')

In [3]:
spec_df = pd.read_pickle('data/specific_returns_halflife36_TOP2000.pkl')
spec_est = spec_df.ewm(halflife=36).var()[spec_df.notna()]
spec_est = spec_est.stack()
spec_est.to_pickle(f'data/spec_est_halflife{36}_TOP{2000}.pkl')


FileNotFoundError: [Errno 2] No such file or directory: 'data/specific_returns_halflife36_TOP2000.pkl'

Function to compute risk model covariance matrix

In [5]:
universe = pd.read_csv(f'data/TOP2000_universe.csv',parse_dates=[0],index_col=0)
ff_loadings = pd.read_pickle('data/ffloadings_halflife36_TOP2000.pkl')
ff_cov_df = pd.read_pickle(f'data/ff_cov_halflife{36}.pkl')
spec_est = pd.read_pickle(f'data/spec_est_halflife{36}_TOP{2000}.pkl')

def risk_cov(date):
    ff_cols = ['Mkt-RF','HML','SMB']
    betas = ff_loadings.loc[date,(universe.loc[date],slice(None))].unstack()[ff_cols].dropna(axis=0)
    ff_cov = ff_cov_df.loc[date].loc[ff_cols,ff_cols]
    spec_cov = spec_est.loc[date]
    
    idx = spec_cov.index.intersection(betas.index)
    spec_cov = spec_cov.reindex(idx)
    betas = betas.reindex(idx)

    cov = betas@ff_cov@betas.T + np.diag(spec_cov)
    return cov

In [16]:
risk_cov(ff_loadings.index[300])

permno,10060,10065,10071,10078,10085,10104,10107,10108,10114,10119,...,90100,90369,90537,90609,90879,91003,91380,91679,92284,92655
permno,,,,,,,,,,,,,,,,,,,,,
10060,0.019621,-0.000337,-0.000716,-0.000003,0.000067,0.000444,-0.000587,0.000520,0.000459,-0.000658,...,-0.000286,-0.000748,0.000302,-0.001066,-0.000698,-0.000585,-0.000494,0.000517,-0.000124,-0.000549
10065,-0.000337,0.001285,0.002246,0.001251,0.000331,0.001640,0.001151,0.000648,0.002182,0.001450,...,0.000924,0.001205,0.000269,0.001546,0.002397,0.001757,0.001566,0.001163,0.002011,0.001561
10071,-0.000716,0.002246,0.016852,0.006455,0.001442,0.007883,0.005320,0.003334,0.010766,0.005745,...,0.003710,0.004808,0.001447,0.006742,0.011091,0.008507,0.007181,0.006682,0.008970,0.007209
10078,-0.000003,0.001251,0.006455,0.015518,0.000838,0.005420,0.003899,0.002352,0.007784,0.003143,...,0.002021,0.002783,0.001044,0.004661,0.007862,0.006483,0.005041,0.005705,0.005749,0.005146
10085,0.000067,0.000331,0.001442,0.000838,0.004743,0.001461,0.000566,0.000728,0.001816,0.000944,...,0.000667,0.000665,0.000338,0.000682,0.001469,0.000961,0.000951,0.000936,0.001554,0.000906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91003,-0.000585,0.001757,0.008507,0.006483,0.000961,0.006399,0.005338,0.002538,0.009396,0.004257,...,0.002631,0.003932,0.001077,0.006610,0.010347,0.035490,0.006660,0.006905,0.007162,0.006850
91380,-0.000494,0.001566,0.007181,0.005041,0.000951,0.005613,0.004142,0.002319,0.007901,0.003924,...,0.002494,0.003414,0.000999,0.005186,0.008361,0.006660,0.015294,0.005298,0.006331,0.005477
91679,0.000517,0.001163,0.006682,0.005705,0.000936,0.006187,0.004013,0.002861,0.008842,0.003011,...,0.002006,0.002605,0.001310,0.004575,0.008312,0.006905,0.005298,0.039857,0.006201,0.005397


In [ ]:
all(np.linalg.eig(risk_cov(ff_loadings.index[100]))[0] > 0)

Risk Model R2

In [ ]:
spec_df = pd.read_pickle('data/specific_returns_halflife36_TOP2000.pkl').stack()
spec_df.index = spec_df.index.set_names('date',level=0)
sys_df = pd.read_pickle('data/systematic_returns_halflife36_TOP2000.pkl').stack()
sys_df.index = sys_df.index.set_names('date',level=0)
spec_df,sys_df = spec_df.align(sys_df,join='inner')

ret_df = spec_df + sys_df




In [ ]:
sns.histplot(ret_df,binrange=(-0.5,0.5))

In [ ]:
from scipy.stats import ttest_1samp

r2_df = 1 - spec_df.clip(-0.5,0.5).groupby('date').var()/ret_df.clip(-0.5,0.5).groupby('date').var()
r2_df.plot(ylim=(-0.5,1))
plt.show()
sns.histplot(r2_df,binrange=(-0.5,1))
plt.xlabel("R2")
plt.show()

print("Mean R2:", r2_df.mean())
print("Median R2:", r2_df.median())
print("T-test:", ttest_1samp(r2_df,0,alternative='greater'))